In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#베이스 모델과는 다른 방식으로 훈련데이터 테스트 데이터로 나누어준후 전처리 진행함


In [ ]:
from glob import glob
path = '/content/drive/MyDrive/animals'
dog = sorted(glob(path+'/dog' + '/*'))
lion = sorted(glob(path+'/lion' + '/*'))
print('dog 이미지 개수: {}\nlion 이미지 개수: {}'.format(len(dog), len(lion)))

In [ ]:
import math

#비율이 적을 test먼저 분할 진행

dog_test= round(len(dog)*0.2)
lion_test = round(len(lion)*0.2)

print('dog test파일에 들어갈 이미지 개수 : {}/{}'.format(dog_test,len(dog)))
print('lion test파일에 들어갈 이미지 개수 : {}/{}'.format(lion_test,len(lion)))

In [ ]:
import shutil
import random
def split(img_list, test_count, train_path, test_path):
    
    test_files=[]
    for i in random.sample( img_list, test_count):
        test_files.append(i)
    
    # 차집합방법으로 train/test 리스트 생성하기
    train_files = [x for x in img_list if x not in test_files]

    for k in train_files:
        shutil.copy(k, train_path)
    
    for c in test_files:
        shutil.copy(c, test_path)
    
    print('train 폴더 이미지 개수 : {}\ntest 폴더 이미지 개수: {}'.format(len(glob(train_path+'/*')),len(glob(test_path+'/*'))))

In [ ]:
#개와 사자 파일
dog_train_path = '/content/drive/MyDrive/train/dog'
dog_test_path = '/content/drive/MyDrive/test/dog' 

lion_train_path = '/content/drive/MyDrive/train/lion'
lion_test_path = '/content/drive/MyDrive/test/lion'

#분할
split(dog, dog_test, dog_train_path, dog_test_path)
split(lion, lion_test, lion_train_path,lion_test_path)

## train/test로 분할을 한 이미지들은 랜덤으로 추출했기때문에 다시 000부터 넘버링해줌.

In [ ]:
import os
def rename(files):

  if 'dog' in files[0]:
     for i,f in enumerate(files):
         os.rename(f, os.path.join(path+"/dog", 'dog_' + '{0:03d}.jpg'.format(i)))
     dog = glob(path+"/dog" + '/*')    
     print("dog {}번째 이미지까지 성공".format(i+1))

  elif 'lion' in files[0]:
     for i,f in enumerate(files):
         os.rename(f, os.path.join(path+"/lion", 'lion_' + '{0:03d}.jpg'.format(i)))
     lion = glob(path+"/lion"+'/*')
     print("lion {}번째 이미지까지 성공".format(i+1))

In [ ]:
#먼저 훈련데이터 해줌.

path = '/content/drive/MyDrive/train'
dog = glob(path+'/dog'+'/*')
lion = glob(path+'/lion'+'/*')

rename(dog)
rename(lion)

In [ ]:
path = '/content/drive/MyDrive/train'
dog = glob(path+'/dog'+'/*')
lion = glob(path+'/lion'+'/*')

print('각각 마지막 파일 이름 : ',os.path.basename(sorted(dog)[-1]), os.path.basename(sorted(lion)[-1]))

In [ ]:
#test파일 넘버링

path = '/content/drive/MyDrive/test'
dog = glob(path+'/dog'+'/*')
lion = glob(path+'/lion'+'/*')

rename(dog)
rename(lion)

In [ ]:
path = '/content/drive/MyDrive/test'
dog = glob(path+'/dog'+'/*')
lion = glob(path+'/lion'+'/*')

print('각각 마지막 파일 이름 : ',os.path.basename(sorted(dog)[-1]), os.path.basename(sorted(lion)[-1]))

In [ ]:
#경로 지정
train_path='/content/drive/MyDrive/train'
test_path ='/content/drive/MyDrive/test'
dog ='/dog/'
lion = '/lion/'

In [ ]:
#이걸 실행하는것은 이것떄문에 클래스가 3개로 인식이되어짐.
#rmdir /content/drive/MyDrive/train/.ipynb_checkpoints

In [ ]:
#rmdir /content/drive/MyDrive/test/.ipynb_checkpoints

In [ ]:
import tensorflow as tf
class_names=['dog','lion']
resize_train= tf.keras.preprocessing.image_dataset_from_directory(
    train_path,
    class_names=class_names,
    color_mode="rgb",
    image_size=(128, 128)
)
resize_test = tf.keras.preprocessing.image_dataset_from_directory(
    test_path,
    class_names=class_names,
    color_mode="rgb",
    image_size = (128,128)
)

In [ ]:
class_names = resize_train.class_names
print(class_names)

In [ ]:
for image_batch  in resize_train:
    print(image_batch[0][0])

In [ ]:
#이미지 성능 향상을 위해서 사용하는것입니다.
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = resize_train.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_dataset = resize_test.cache().prefetch(buffer_size=AUTOTUNE)
#데이터셋 객체에 캐싱이나 셔플 프리배치와 같은 이미지 입력 관련 성능 향상해준다.

In [ ]:
#시각화 진행
import matplotlib.pyplot as plt
import numpy 
for images, labels in train_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i+1 )
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

## 데이터 전처리 진행

In [ ]:
#데이터 증강을 레이어로 만들어서 나중에 모델에 추가할 예정입니다.
#각 순전파마다 자동으로 데이터가 증강이되며, 매번 학습이 될때마다 다른 이미지로 학습 하는 결과를 출력.
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)
])


In [ ]:
#증강된 이미지 시각화로 확인
for image, _ in train_dataset.take(1):
    plt.figure(figsize=(10,10))
    first_image = image[0]
    for i in range(9):
        ax = plt.subplot(3, 3, i+1)
        augmented_image = data_augmentation(tf.expand_dims(first_image, 0 ))
        plt.imshow(augmented_image[0] / 255) #normalization도 진행
        plt.axis('off')
#출력될때마다 다른 이미지 9개를 출력하는걸 알수있음.

In [ ]:
#이미지 사이즈를 0~255,0~1로 할지 정하기
preprocess_input = tf.keras.layers.experimental.preprocessing.Rescaling(scale=1./255)

In [ ]:
#이미지 인식을 위해 특징을 추출하는 모델을 사용합니다.
import tensorflow_hub as hub
feature_extractor_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"
IMG_SIZE = (128,128)
IMG_SHAPE = IMG_SIZE + (3,)
#새로운 모델을 만듭니다.
first_model = hub.KerasLayer(feature_extractor_url, input_shape=IMG_SHAPE)

In [ ]:
#출력값 확인
image_batch, label_batch = next(iter(train_dataset))
feature_batch = first_model(image_batch)
print(feature_batch)

In [ ]:
first_model.trainable = False

##모델 조립

In [ ]:
#특징 추출에서 나온것을 받아보기위해서 몇개의 클래스를 반환하는 fc레이어를 제작합니다.
prediction_layer = tf.keras.layers.Dense(len(class_names))

In [ ]:
inputs = tf.keras.Input(shape=IMG_SHAPE)
x = data_augmentation(inputs)
x = preprocess_input(x)
x = first_model(x, training=False)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model= tf.keras.Model(inputs, outputs)

In [ ]:
model.summary()

In [ ]:
#모델 컴파일
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
#학습 진행
initial_epochs = 10
history = model.fit(
    train_dataset,
    validation_data = test_dataset,
    epochs= initial_epochs
)

In [ ]:
#시각화 진행
accuracy = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(10, 10))
plt.subplot(2, 1, 1)
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entorpy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

### 학습할 시간이 부족해서 적게 학습을 진행하였지만 학습을 할수록 모델의 성능이 좋아지고 있는걸 시각화를 통해서 확인했으며,
- 모델의 성능이 괜찮게 나오는걸 확인을 할수있었습니다.
- 2개의 클래스를 통해서 이미지 분류가 확실히 잘되었으므로
- 이미지 인식을 하는 서비스를 진행할것입니다.


## 그전에 파인 튜닝을 해서 모델을 학습 시켜서 시각화를 진행하겠습니다.

In [ ]:
#파인튜닝을 했을때
first_model.trainable = True

fine_tune_epochs =10
total_epochs = initial_epochs + fine_tune_epochs

history_fine = model.fit(train_dataset,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=test_dataset
)

In [ ]:
accuracy  += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

#파인튜닝을 했을때 모델의 결과 시각화

plt.figure(figsize=(10,10))
plt.subplot(2, 1, 1)
plt.plot(accuracy, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.8, 1])
plt.plot([initial_epochs-1, initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
#만든 모델 저장하기
from tensorflow.python.keras.models import load_model
model.save('mnist_mlp_model.h5')

## 이미지데이터의 증강을 통해서 모델의 성능이 좋아졌지만
### 파인튜닝을 통해서 더좋은 성능을 보여주고있는모습입니다.

